Airplane_Accident

Problem statement
Flying has been the go-to mode of travel for years now; it is time-saving, affordable, and extremely convenient. According to the FAA, 2,781,971 passengers fly every day in the US, as in June 2019. Passengers reckon that flying is very safe, considering strict inspections are conducted and security measures are taken to avoid and/or mitigate any mishappenings. However, there remain a few chances of unfortunate incidents.

Imagine you have been hired by a leading airline. You are required to build Machine Learning models to anticipate and classify the severity of any airplane accident based on past incidents. With this, all airlines, even the entire aviation industry, can predict the severity of airplane accidents caused due to various factors and, correspondingly, have a plan of action to minimize the risk associated with them.

Data set
The dataset consists of certain parameters recorded during the incident⁠ such as cabin temperature, turbulence experienced, number of safety complaints prior to the accident, number of days since the last inspection was conducted before the incident, an estimation of the pilot’s control given the various factors at play, and the likes.
The benefits of practicing this problem by using Machine Learning techniques are as follows:
This challenge will encourage you to apply your Machine Learning skills to build models that can anticipate the severity of any airplane accident
This challenge will help you enhance your knowledge of classification actively. Classification is one of the basic building blocks of Machine Learning
We challenge you to build a model that predicts how severe an airplane accident could be.

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


Importing all the libraries


In [6]:
import os
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
# !pip install seaborn
import seaborn as sns

#!pip install imblearn
#if the above command does not work to install imblearn package run the following command in your terminal
# conda install -c glemaitre imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, recall_score, precision_score

import warnings
warnings.filterwarnings('ignore')

Reading the data sets

In [7]:
test_data = pd.read_csv('/gdrive/MyDrive/Kaggle data sets/Airplane_Accident_Master/test.csv', sep=',', header=0, na_values = 'unknown')
train_data = pd.read_csv('/gdrive/MyDrive/Kaggle data sets/Airplane_Accident_Master/train.csv', sep=',', header=0, na_values = 'unknown')

In [8]:
test_data.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,19.497717,16,6,72.151322,0.388959,78.32,4,37949.724386,2,0.069692,1
1,58.173516,15,3,64.585232,0.250841,78.60,7,30194.805567,2,0.002777,10
2,33.287671,15,3,64.721969,0.336669,86.96,6,17572.925484,1,0.004316,14
3,3.287671,21,5,66.362808,0.421775,80.86,3,40209.186341,2,0.199990,17
4,10.867580,18,2,56.107566,0.313228,79.22,2,35495.525408,2,0.483696,21


In [9]:
train_data.head()

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [10]:
test_data.dtypes

Safety_Score               float64
Days_Since_Inspection        int64
Total_Safety_Complaints      int64
Control_Metric             float64
Turbulence_In_gforces      float64
Cabin_Temperature          float64
Accident_Type_Code           int64
Max_Elevation              float64
Violations                   int64
Adverse_Weather_Metric     float64
Accident_ID                  int64
dtype: object

In [11]:
train_data.dtypes

Severity                    object
Safety_Score               float64
Days_Since_Inspection        int64
Total_Safety_Complaints      int64
Control_Metric             float64
Turbulence_In_gforces      float64
Cabin_Temperature          float64
Accident_Type_Code           int64
Max_Elevation              float64
Violations                   int64
Adverse_Weather_Metric     float64
Accident_ID                  int64
dtype: object

In [12]:
train_data.describe()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000
mean,41.876406,12.931100,6.564300,65.145324,0.381495,79.969331,3.814900,32001.803282,2.01220,0.255635,6266.554200
std,16.138072,3.539803,6.971982,11.882934,0.121301,2.759739,1.902577,9431.995196,1.03998,0.381128,3610.170288
min,0.000000,1.000000,0.000000,0.000000,0.134000,74.740000,1.000000,831.695553,0.00000,0.000316,2.000000
25%,30.593607,11.000000,2.000000,56.927985,0.293665,77.960000,2.000000,25757.636908,1.00000,0.012063,3139.750000
50%,41.278539,13.000000,4.000000,65.587967,0.365879,79.540000,4.000000,32060.336419,2.00000,0.074467,6280.500000
75%,52.511416,15.000000,9.000000,73.336372,0.451346,81.560000,5.000000,38380.641513,3.00000,0.354059,9391.500000
max,100.000000,23.000000,54.000000,100.000000,0.882648,97.510000,7.000000,64297.651218,5.00000,2.365378,12500.000000


In [13]:
print(list(train_data.columns))

['Severity', 'Safety_Score', 'Days_Since_Inspection', 'Total_Safety_Complaints', 'Control_Metric', 'Turbulence_In_gforces', 'Cabin_Temperature', 'Accident_Type_Code', 'Max_Elevation', 'Violations', 'Adverse_Weather_Metric', 'Accident_ID']


In [14]:
train_data.describe(include='object')

,Severity
count,10000
unique,4
top,Highly_Fatal_And_Damaging
freq,3049


In [16]:
train_data.Severity.value_counts()

Highly_Fatal_And_Damaging                  3049
Significant_Damage_And_Serious_Injuries    2729
Minor_Damage_And_Injuries                  2527
Significant_Damage_And_Fatalities          1695
Name: Severity, dtype: int64

In [18]:
train_data.groupby('Severity').mean()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
Severity,,,,,,,,,,,
Highly_Fatal_And_Damaging,33.353611,12.764513,6.337160,66.541401,0.367432,80.023526,3.467366,31940.145431,1.963267,0.302307,6307.969170
Minor_Damage_And_Injuries,47.800305,13.170162,6.827859,64.350232,0.386255,79.903961,3.269490,32225.568306,1.981797,0.311014,6292.467353
Significant_Damage_And_Fatalities,47.102113,13.071386,6.486726,60.059293,0.412759,80.041516,4.500885,31300.789137,2.175811,0.247845,6181.897345
Significant_Damage_And_Serious_Injuries,42.667443,12.808721,6.622206,67.480750,0.373382,79.924478,4.282155,32298.893626,1.993404,0.157048,6248.868816


In [19]:
train_data.Severity.value_counts()

Highly_Fatal_And_Damaging                  3049
Significant_Damage_And_Serious_Injuries    2729
Minor_Damage_And_Injuries                  2527
Significant_Damage_And_Fatalities          1695
Name: Severity, dtype: int64

Type Casting

In [20]:
for col in ['Severity']:
  train_data[col] = train_data[col].astype("category")


In [21]:
train_data.dtypes

Severity                   category
Safety_Score                float64
Days_Since_Inspection         int64
Total_Safety_Complaints       int64
Control_Metric              float64
Turbulence_In_gforces       float64
Cabin_Temperature           float64
Accident_Type_Code            int64
Max_Elevation               float64
Violations                    int64
Adverse_Weather_Metric      float64
Accident_ID                   int64
dtype: object

In [22]:
cat_attr = list(train_data.select_dtypes("category").columns)
num_attr = list(train_data.columns.difference(cat_attr))
cat_attr.pop()

'Severity'

In [23]:
train_data.isnull().sum()

Severity                   0
Safety_Score               0
Days_Since_Inspection      0
Total_Safety_Complaints    0
Control_Metric             0
Turbulence_In_gforces      0
Cabin_Temperature          0
Accident_Type_Code         0
Max_Elevation              0
Violations                 0
Adverse_Weather_Metric     0
Accident_ID                0
dtype: int64

In [24]:
numeric_transformer = Pipeline(steps=[
                                      ('Imputer', SimpleImputer(strategy = "median")),
                                      ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_attr),
        ('cat', categorical_transformer, cat_attr)])

In [25]:
clf_logreg = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

Train_Test Split

In [26]:
y = train_data['Severity']
x = train_data.drop(['Severity'],axis = 1)

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 123)

In [27]:
X_train.head()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
3144,40.639269,9,3,41.020966,0.543304,80.15,1,34325.615077,3,1.263655,8159
9939,18.310502,16,9,59.070191,0.392565,78.29,5,23487.729496,3,0.014991,1408
7925,38.812785,13,0,61.531449,0.424299,86.39,3,831.695553,3,0.047601,100
309,32.876712,15,7,57.748405,0.478392,78.85,5,33635.234643,2,0.023609,337
9415,28.219178,13,10,86.964448,0.289427,80.65,4,36065.208527,2,0.065909,7196


build  logistic regression model

In [28]:
clf_logreg.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('Imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                             

Evaluate model

In [32]:
train_pred = clf_logreg.predict(X_train)
test_pred = clf_logreg.predict(X_test)

print("Train_pred",clf_logreg.score(X_train, y_train))
print("Test_pred",clf_logreg.score(X_test, y_test))

score = 100*(f1_score(y_true = y_train, y_pred = train_pred, average="weighted"))
print("F1_score:", score)

Train_pred 0.6454285714285715
Test_pred 0.6263333333333333
F1_score: 63.40572244827799


build Decision tree model

In [33]:
%%time
clf_dt = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', DecisionTreeClassifier())])

dt_param_grid = {'classifier__criterion': ['entropy', 'gini'], 'classifier__max_depth': [6,8,10,12], 
                 "classifier__min_samples_split": [2, 10, 20],"classifier__min_samples_leaf": [1, 5, 10]}

dt_grid = GridSearchCV(clf_dt, param_grid=dt_param_grid, cv=5)

dt_grid.fit(X_train,y_train)

CPU times: user 27.8 s, sys: 79.9 ms, total: 27.9 s
Wall time: 27.9 s


In [34]:
dt_grid.best_params_

{'classifier__criterion': 'entropy',
 'classifier__max_depth': 12,
 'classifier__min_samples_leaf': 1,
 'classifier__min_samples_split': 2}

In [35]:
train_pred = dt_grid.predict(X_train)
test_pred = dt_grid.predict(X_test)

print(dt_grid.score(X_train, y_train))
print(dt_grid.score(X_test, y_test))

0.9891428571428571
0.9336666666666666


In [36]:
score_1 = 100*(f1_score(y_true = y_train, y_pred = train_pred, average = 'weighted'))
print("F1_score :",score_1)

F1_score : 98.91530706099607


Build Random Forest Model - 3_ (Using Stratified KFold)

__Stratified K-Folds cross-validator__

This cross-validation object is a **variation** of KFold that returns stratified folds. The folds are made by **preserving the percentage of samples for each class**.

In [39]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])


In [40]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=143)

param_grid = {"classifier__n_estimators" : [150, 250, 300],
              "classifier__max_depth" : [5,8,10],
              "classifier__max_features" : [3, 5, 7],
              "classifier__min_samples_leaf" : [4, 6, 8, 10]}

rf_grid = GridSearchCV(clf, param_grid= param_grid, cv=kfold)

In [41]:
%%time
rf_grid.fit(X_train,y_train)

CPU times: user 25min 27s, sys: 1.86 s, total: 25min 28s
Wall time: 25min 35s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=143, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('Imputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                

In [42]:
rf_grid.best_params_

{'classifier__max_depth': 10,
 'classifier__max_features': 7,
 'classifier__min_samples_leaf': 4,
 'classifier__n_estimators': 150}

In [43]:
train_pred = rf_grid.predict(X_train)
test_pred = rf_grid.predict(X_test)

print(rf_grid.score(X_train, y_train))
print(rf_grid.score(X_test, y_test))

score_2 = 100*(f1_score(y_true = y_train, y_pred = train_pred, average = 'weighted'))
print("F1_score :",score_2)

0.9597142857142857
0.9226666666666666
F1_score : 95.97875521188777


Build Gradient Boosting - 4

In [44]:
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('GBM',GradientBoostingClassifier())])

In [45]:
%%time
gbm_param_grid = {'GBM__max_depth': [8,10,12,14], 'GBM__subsample': [0.8, 0.6,], 'GBM__max_features':[0.2, 0.3], 
              'GBM__n_estimators': [10, 20, 30]}

gbm_grid = GridSearchCV(clf, param_grid=gbm_param_grid, cv=3)

gbm_grid.fit(X_train,y_train)

CPU times: user 5min 5s, sys: 102 ms, total: 5min 5s
Wall time: 5min 6s


In [46]:
gbm_grid.best_params_

{'GBM__max_depth': 10,
 'GBM__max_features': 0.3,
 'GBM__n_estimators': 30,
 'GBM__subsample': 0.8}

In [47]:
train_pred = gbm_grid.predict(X_train)
test_pred = gbm_grid.predict(X_test)

print(gbm_grid.score(X_train, y_train))
print(gbm_grid.score(X_test, y_test))

score_3 = 100*(f1_score(y_true = y_train, y_pred = train_pred, average = 'weighted'))
print("F1_score :",score_3)

0.9998571428571429
0.906
F1_score : 99.98571410501515


In [48]:
rf1 = RandomForestClassifier(n_estimators=10)

In [49]:
rf1.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [50]:
y_pred=rf1.predict(X_test)

In [51]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score


In [52]:
y_train_pred=rf1.predict(X_train)
print("F1_Score for Train:",f1_score(y_train,y_train_pred,average='macro'))

F1_Score for Train: 0.9973643218707838


In [53]:
print("Accuracy :",accuracy_score(y_test,y_pred))

Accuracy : 0.8403333333333334


In [54]:
sub=rf1.predict(test_data)

In [55]:
final= pd.DataFrame({'Accident_ID' : test_data['Accident_ID'],
                     'Severity' : sub})

In [56]:
final.to_csv("submission_copy2.csv",index=False)
final.shape

(2500, 2)